In [ ]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.action_chains import ActionChains
import winsound

In [ ]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
url_template_pages = "https://www.pets4homes.co.uk/{}/{}/local/{}/page-{}/"
listing_type = ['sale','adoption','stud']
pet_type = ['dogs','cats','reptiles','rodents','rabbits','horses','invertebrates','livestock', 'poultry', 'birds','fish']
#cities = ['aberdeen','ayr_st-ives', 'barnsley_wimborne','basingstoke','bath','bedfordshire','birmingham','blackpool_plymouth',
#         'bolton_appleby-in-westmorland', 'bournemouth', 'bradford_holsworthy', 'bridgend_brechin', 'brighton',
#          'bristol_south-west-england', 'cambridge_gloucester', 'cardiff-county', 'carlisle', 'chelmsford', 'cheltenham',
#         'chester']
cities = ['aberdeen', 'basingstoke',
          'bournemouth', 'bridgend_brechin',
          'bristol_south-west-england', 'chelmsford',
         'chester']

In [ ]:
def check_captcha (soup):
    if 'I am human' in soup.get_text():
            print('captcha detected')
            duration = 5000  # milliseconds
            freq = 440  # Hz
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
    else:
        print('no captcha detected, proceed to scrape')

In [ ]:
def scrape_header(soup):
    df = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
    for each in soup.find_all("script", type="application/ld+json"):
        entries = each.get_text()
        split = entries.split("\n")
        Title = split[3].replace('"name": "', '').replace('",', '')
        Description = split[4].replace('"description": "', '').replace('",', '').replace('\n,', '')
        Photo = split[5].replace('"image": "', '').replace('",', '')
        Category = split[7].replace('"category": "', '').replace('",', '')
        Price = split[11].replace('"price": "', '').replace('",', '')
        url =split[13].replace('"url": "', '').replace('",', '')
        seller_type = split[15].replace('"type": "', '').replace('",', '')
        seller_name = split[16].replace('"name":  "', '').replace('"', '')
        df = df.append({"Title":Title,"Description":Description,"photo_link":Photo,"category":Category, 
                                      "price":Price,"url":url, "seller_type":seller_type, "seller_name":seller_name,
                                      "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df

In [ ]:
def scrape_content(soup):
    df1 = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                "seller_name", "seller_location", "seller_type", "listing_type","pet_type"])
    for each in soup.find_all("div", class_="Xm lj"):
        entries = each.get_text()
        title = each.find('h2', {'class':'gn'}).text
        price = each.find('span', {'class':'hn'}).text
        info = each.find_all('span', {'class':'nv'})
        species = info[0].text
        try:
            age = info[1].text.replace('Age:  ', '')
        except:
            age = 'unknown'
        try:
            gender = info[2].text
        except:
            gender = 'unknown'
        description = each.find('span', {'class':'in'}).text
        seller_name = each.find('span', {'class':'on'}).text
        seller_location = each.find('span', {'class':'gs'}).text
        seller_type = each.find('div', {'class':'ef Pl ff'}).text
        df1 = df1.append({"Title":title, "price":price, "species":species, "age":age, "gender":gender,
                              "description":description, "seller_name":seller_name, "seller_location": seller_location,
                              "seller_type":seller_type, "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df1

In [ ]:
completed = []
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            page = 1
            while page > 0:
                clear_output(wait=True)
                file = 'data_by_cities/header_'+str(city)+'.csv'
                file1 = 'data_by_cities/body_'+str(city)+'.csv'
                try: 
                    df_header = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_header = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
                    df_header.to_csv(file, sep='\t', encoding='utf-8')
                try: 
                    df_body = pd.read_csv(file1 , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_body = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                                    "seller_name", "seller_location", "seller_type" , 
                                                    "listing_type","pet_type"])
                    df_body.to_csv(file1, sep='\t', encoding='utf-8')
                headersize = df_header.shape
                bodysize = df_body.shape
                if page == 1:
                    url = url_template.format(listing, pets, city)
                else:
                    url = url_template_pages.format(listing, pets, city, page)
                options = webdriver.ChromeOptions()
                browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
                browser.get(url)
                title = (
                    WebDriverWait(driver=browser, timeout=15)
                    .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
                )
                content = browser.page_source
                print(city,pets,listing,page)
                soup = BeautifulSoup(content, 'lxml')
                check_captcha(soup)
                if 'We found 0' in soup.get_text():
                    print('no result found')
                    break
                df_header = df_header.append(scrape_header(soup))
                df_body = df_body.append(scrape_content(soup))
                #drop duplicates
                df_header = df_header[~df_header.duplicated(subset=['Title','Description', 'photo_link','category','url','seller_type','listing_type','pet_type'])].reset_index(drop=True)
                df_header.to_csv(file, sep='\t', encoding='utf-8')
                df_body = df_body[~df_body.duplicated()].reset_index(drop=True)
                df_body.to_csv(file1, sep='\t', encoding='utf-8')
                if df_body.shape == bodysize:
                    if df_header.shape == headersize:
                        print('no new data')
                    else:
                        winsound.Beep(600, 1000)
                        input('html tags is likely changed, please kill program')
                next_page = soup.find('a', {'class':'Gb Xg Kb'})
                if next_page == None:
                    print('next page not found')
                    break
                else:
                    print ('next page found')
                    print('going to next page')
                    sleep(random.randint(10,15))
                    page +=1
            print('going to next listing type')
            sleep(random.randint(10,15))
        print('going to next pet type')
    completed.append(city)
    print('going to next city')

In [ ]:
completed